<a href="https://colab.research.google.com/github/ericmcg13/CCT-CA1/blob/main/CA1_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

'Dash' is a required install for use of the plotly graphics and sliders


In [1]:
!pip install dash
!pip install ipyleaflet

     |████████████████████████████████| 7.3 MB 8.2 MB/s 
     |████████████████████████████████| 23.9 MB 12 kB/s 
     |████████████████████████████████| 357 kB 55.1 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=9c604360ae0243e12354939e6820e19f9115cbee209b2ac0e60999a9e15050ee
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=bfd2e4ddb396e3b6a7a85ec9ed59f34775e8df07299511a8e82be1af69f03fac
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=1831204a9b95ab6d82f8a39ebfa42a59b9bfa33d5945efe84389e09ac95f3c8b
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aefd

In [4]:
import plotly.express as px
import pandas as pd
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles

In [6]:
deaths_df = pd.read_csv('/content/DATA_FILES/Deaths_Region_2007-2020.csv')
deaths_df.head()

,Statistic,Year,Area of Residence,Sex,Age at Death,UNIT,VALUE
0,Deaths Occurring,2007,ROI Total,Both sexes,Under 1 year,Number,230.0
1,Deaths Occurring,2007,ROI Total,Both sexes,1 - 4 years,Number,54.0
2,Deaths Occurring,2007,ROI Total,Both sexes,5 - 9 years,Number,17.0
3,Deaths Occurring,2007,ROI Total,Both sexes,10 - 14 years,Number,50.0
4,Deaths Occurring,2007,ROI Total,Both sexes,15 - 19 years,Number,149.0


In [5]:
# Map code / ignore until data is ready

# center = [53.128, -7.5]
# zoom = 6

# # Create a map with the defined aspects.
# m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
# m

Map(center=[53.128, -7.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [8]:
clean = deaths_df.dropna(how='any')
clean['index'] = clean.index
clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71149 entries, 0 to 123059
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Statistic          71149 non-null  object 
 1   Year               71149 non-null  int64  
 2   Area of Residence  71149 non-null  object 
 3   Sex                71149 non-null  object 
 4   Age at Death       71149 non-null  object 
 5   UNIT               71149 non-null  object 
 6   VALUE              71149 non-null  float64
 7   index              71149 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 4.9+ MB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
# Scatter is not loaded with correct data, only a POC

import plotly.express as px

fig = px.scatter(clean, x="index", y="VALUE", animation_frame="Year", animation_group="Area of Residence",
           size="VALUE", color="Area of Residence",
           log_x=True, size_max=1000, range_x=[100,100000], range_y=[1,50])

fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()